In [ ]:
import os
os.listdir('/kaggle/input')

In [ ]:
print(os.listdir('/kaggle/input/new-plant-diseases-dataset/'))

In [ ]:
print(os.listdir('/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'))
print(os.listdir('/kaggle/input/plant-pathology-2020-fgvc7'))

In [ ]:
print(os.listdir('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/'))

In [ ]:
import math
import random
import itertools

import numpy as np
import pandas as pd

import random
from sklearn.utils import shuffle
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

# fastai 1.0
from fastai import *
from fastai.vision import *
from torchvision.models import *   

import re
import cv2

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
SEED = 300
seed_everything(SEED)

In [ ]:
work_dir1 = '/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
len(os.listdir('/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'))

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
def getEfficientNet(model_name,pretrained=True):
    """Constructs a EfficientNet model for FastAI.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = EfficientNet.from_pretrained(model_name)
    model._fc == nn.Linear(model._fc.in_features, 38)
    return model

**Pretraining**

In [ ]:
imgDataBunch = (ImageDataBunch.from_folder(path=work_dir1, 
                                              size=224, 
                                              bs=32, 
                                              ds_tfms=get_transforms())

                  .normalize())

In [ ]:
imgDataBunch.show_batch(rows=3, figsize=(8,8))

In [ ]:
learn = Learner(imgDataBunch, getEfficientNet('efficientnet-b4'),
               metrics = error_rate,
               callback_fns=ShowGraph).mixup().to_fp16()

In [ ]:
learn.split(lambda m: (m._conv_head,))
learn.freeze()

In [ ]:
wd = 1e-1
learn.model_dir = '/tmp/model/'
learn.lr_find(wd=wd)
learn.recorder.plot(suggestion = True)

In [ ]:
max_lr = 1.74e-02

In [ ]:
learn.fit_one_cycle(cyc_len=5, max_lr=max_lr, wd=wd)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(imgDataBunch.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60)

In [ ]:
learn.unfreeze()
learn.lr_find(wd=wd)

learn.recorder.plot(suggestion = True)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

In [ ]:
#learn.save('/kaggle/working/model_1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(imgDataBunch.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60)

In [ ]:
classes = os.listdir('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/')

In [ ]:
classes.sort()

In [ ]:
classes

In [ ]:
n_train = []
n_valid = []
for i in classes:
    n_train.append(len(os.listdir(f'/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/{i}')))
    n_valid.append(len(os.listdir(f'/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid/{i}')))

In [ ]:
df1 = pd.DataFrame(data = {'train_img_n': n_train, 'valid_img_n': n_valid})

In [ ]:
cm = interp.confusion_matrix()

In [ ]:
precision = []
recall = []
for i in range(cm.shape[0]):
    recall.append(cm[i,i]/np.sum(cm[i,:]))
    precision.append(cm[i,i]/np.sum(cm[:,i]))

f1 = [precision[i]*recall[i] for i in range(len(recall))]

In [ ]:
df3 = pd.DataFrame(data = {'precision': precision, 'recall': recall, 'f1': f1})

In [ ]:
df3.round(4).to_csv('/kaggle/working/df3.csv')

In [ ]:
imgDataBunch.show_batch(rows=1, figsize=(8,8))

In [ ]:
from fastai.callbacks.hooks import *

In [ ]:
def GradCAM(idx:int,interp:ClassificationInterpretation, image = True):
    m = interp.learn.model.eval()
    im,cl = interp.learn.data.dl(DatasetType.Valid).dataset[idx]
    cl = int(cl)
    xb,_ = interp.data.one_item(im) #put into a minibatch of batch size = 1
    xb_im = Image(interp.data.denorm(xb)[0])
    xb = xb.cuda()
    layer_conv = 3
    mykeys = m._modules.keys()
    k = list(mykeys)
    m0 = m._modules[k[layer_conv]]
    with hook_output(m0) as hook_a: 
        with hook_output(m0, grad=True) as hook_g:
            preds = m(xb)
            preds[0,int(cl)].backward() 
    acts  = hook_a.stored[0].float().cpu() #activation maps
    grad = hook_g.stored[0][0].float().cpu()
    grad_chan = grad.mean(1).mean(1)
    mult = ((acts*grad_chan[...,None,None])).sum(0) #F.relu(((acts*grad_chan[...,None,None])).sum(0))
    if image:
        fig,ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
        sz = list(xb_im.shape[-2:])
        fig.suptitle(f"pred. class: {learn.data.classes[interp.pred_class[idx]]}, actual class: {learn.data.classes[cl]}")
        xb_im.show(ax[0])
        xb_im.show(ax[1])
        ax[0].imshow(mult, alpha=0.8, extent=(0,*sz[::-1],0),
              interpolation='bilinear', cmap='magma')
        ax[1].imshow(mult, alpha=0.0, extent=(0,*sz[::-1],0),
              interpolation='bilinear', cmap='magma')
        plt.subplots_adjust(wspace=0, hspace=0)
    return mult

In [ ]:
_ = GradCAM(np.random.randint(len(learn.data.valid_ds)),interp)